<a href="https://colab.research.google.com/github/MD-Daniel99/Projects/blob/main/team_project_ver_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install sentencepiece sacremoses transformers
!pip install --upgrade python-docx
!pip3 install --upgrade pdfminer.six

from transformers import pipeline
from transformers import AutoTokenizer

from docx import Document
from pdfminer.high_level import extract_text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 52.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 20.2 MB/s eta 0:00:00


## File type recognition

In [2]:
filename = "/content/Black_body_radiation(test.docx"

In [3]:
def filetype_recognition(filename):
  if filename.lower().endswith('.docx'):
    return docx_reader(filename)
  elif filename.lower().endswith('.pdf'):
    return pdf_reader(filename)
  elif filename.lower().endswith('.txt'):
    return txt_reader(filename)
  else:
    raise ValueError("Unsupported file format. Please, make sure you download .docx, .pdf or .txt.")

## .docx reader

In [4]:
def docx_reader(filename):
  doc = Document(filename)
  text = "\n".join([paragraph.text for paragraph in doc.paragraphs])
  str(text)
  return text

## .pdf reader


In [5]:
def pdf_reader(filename):
  text = extract_text(filename)
  str(text)
  return text

## .txt reader

In [6]:
def txt_reader(filename):
  with open(filename, 'r') as file_object:
    text = file_object.read()
    str(text)
  return text

## Summary model start-up

In [7]:
def trim_text_to_token_limit(content, max_token_limit = 512, model_name="Falconsai/text_summarization"):
    # Загрузка токенизатора для заданной модели
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # Токенизация текста
    tokens = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(content)))

    # Обрезка текста до заданного количества токенов
    trimmed_tokens = tokens[:max_token_limit]

    # Обратная токенизация для получения текста
    trimmed_text = tokenizer.decode(tokenizer.convert_tokens_to_ids(trimmed_tokens))

    return trimmed_text

In [18]:
def summary_model(contents, trimmed_text):
  #contents = pipeline('summarization', model = "Falconsai/text_summarization")
  summarised_text = contents(trimmed_text, min_length = 70)[0]
  return summarised_text['summary_text'] # тип данных = строка

## En-Ru translation model start-up

In [9]:
def model_loading():
  classifier = pipeline("translation_en_to_ru", model = "Helsinki-NLP/opus-mt-en-ru")
  contents = pipeline('summarization', model = "Falconsai/text_summarization")
  return classifier, contents

In [10]:
def translation(classifier, summary):
  sentences = summary.split('. ')
  translated_sentences = [classifier(sentence)[0]['translation_text'] for sentence in sentences]
  return '. '.join(translated_sentences)

In [20]:
def printing(translated):
  output_file = '/content/processed_text.txt'
  with open(output_file, 'w') as file:
    file.write(translated)

In [12]:
def file_preprocessing(filename):
    file_content = filetype_recognition(filename)
    return file_content.rstrip()

In [21]:
def main():
  try:
    classifier, contents = model_loading()
    content = file_preprocessing(filename)
    trimmed_text = trim_text_to_token_limit(content)
    summary = summary_model(contents, trimmed_text) # МОДЕЛЬ номер 1 - резюме
    prediction = translation(classifier, summary) # МОДЕЛЬ номер 2 - перевод
    printing(prediction)

  except ValueError as ve:
    print(ve)
    return

if __name__ == "__main__":
    main()

Token indices sequence length is longer than the specified maximum sequence length for this model (516 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (515 > 512). Running this sequence through the model will result in indexing errors


In [22]:
main()

Token indices sequence length is longer than the specified maximum sequence length for this model (516 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (515 > 512). Running this sequence through the model will result in indexing errors
